In [1]:
import numpy as np
import pandas as pd
import json
import re
from collections import Counter
import little_mallet_wrapper
import seaborn
import glob
from pathlib import Path
#LOADING EVERYTHING
directory = "/Users/joshs/Desktop/Spring 2021/HUM346/wsj-nyt-oped/NYTimes"

In [38]:
#Importing stuff from NYTIMES
from pynytimes import NYTAPI
import urllib3, json, datetime
nyt = NYTAPI("nKBpzKBvOYipTxZf1T1NwCsVeXad5xoE", parse_dates=True)
articles = nyt.article_search(
    results = 500,
    dates = {
        "begin": datetime.datetime(2020, 12, 1),
        "end": datetime.datetime(2020, 12, 30)
    },
    options = {
        "sources": [
            "New York Times",
        ],
        "news_desk": [
            "OpEd",
            "Editorial"
            
        ]
    }
)

In [39]:
#Writing articles to be saved
for article in articles:
    article['abstract'] = re.sub('\?', '', article['abstract'])
    article['abstract'] = re.sub('\.', '', article['abstract'])
    article['abstract'] = re.sub('\/', '', article['abstract'])
    text_file = open(f"./Articles/{article['abstract']}.txt", "w+")
    n = text_file.write(article['lead_paragraph'] + " \n", )
    text_file.close()

In [40]:
#splitting text into all lowercase
def split_into_words(any_chunk_of_text):
    lowercase_text = any_chunk_of_text.lower()
    split_words = re.split("\W+", lowercase_text) 
    return split_words

In [41]:
#Combining all data together IT SHOULD WORK NOW BUT WAS NOT WORKING EARLIER
files = glob.glob(f"{directory}/Articles/*.txt")
training_data = []
for file in files:
    text = open(file, encoding='utf-8').read()
    processed_text = little_mallet_wrapper.process_string(text)
    training_data.append(processed_text)

In [38]:
original_texts = []
for file in files:
    text = open(file).read()
    original_texts.append(text)

In [7]:

full_text = training_data
number_of_desired_words = 10
# Manipulate and Analyze File
stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours',
'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers',
 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves',
 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are',
 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does',
 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into',
 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down',
 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here',
 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',
 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so',
 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now', 've', 'll', 'amp']


all_the_words = split_into_words(full_text)
meaningful_words = [word for word in all_the_words if word not in stopwords]
meaningful_words_tally = Counter(meaningful_words)
most_frequent_meaningful_words = meaningful_words_tally.most_common(number_of_desired_words)

AttributeError: 'list' object has no attribute 'lower'

In [6]:
most_frequent_meaningful_words

[('many', 11),
 ('year', 9),
 ('new', 9),
 ('one', 9),
 ('world', 8),
 ('pandemic', 8),
 ('months', 8),
 ('coronavirus', 7),
 ('us', 7),
 ('states', 6)]

In [42]:

path_to_mallet = '/Users/joshs/mallet-2.0.8/bin/mallet'
num_topics = 5
    
#Change to your desired output directory
output_directory_path = '/Users/joshs/Desktop/Spring 2021/HUM346/wsj-nyt-oped/NYTimes/topic-model-output'

#No need to change anything below here
Path(f"{output_directory_path}").mkdir(parents=True, exist_ok=True)

path_to_training_data           = f"{output_directory_path}/training.txt"
path_to_formatted_training_data = f"{output_directory_path}/mallet.training"
path_to_model                   = f"{output_directory_path}/mallet.model.{str(num_topics)}"
path_to_topic_keys              = f"{output_directory_path}/mallet.topic_keys.{str(num_topics)}"
path_to_topic_distributions     = f"{output_directory_path}/{str(num_topics)}"


In [43]:
little_mallet_wrapper.print_dataset_stats(training_data)


Number of Documents: 3595
Mean Number of Words per Document: 26.6
Vocabulary Size: 15500


In [44]:
#Importing Data
little_mallet_wrapper.import_data(path_to_mallet,
                path_to_training_data,
                path_to_formatted_training_data,
                training_data)

Importing data...
Complete


In [ ]:
#Training Data Doesnt seem to work?
little_mallet_wrapper.train_topic_model(path_to_mallet,
                      path_to_formatted_training_data,
                      path_to_model,
                      path_to_topic_keys,
                      path_to_topic_distributions,
                      num_topics)

Training topic model...


In [46]:
topics = little_mallet_wrapper.load_topic_keys(path_to_topic_keys)

for topic_number, topic in enumerate(topics):
    print(f"✨Topic {topic_number}✨\n\n{topic}\n")

✨Topic 0✨

['trump', 'president', 'election', 'donald', 'biden', 'NUM', 'joe', 'political', 'presidential', 'court', 'democratic', 'party', 'state', 'would', 'national', 'last', 'republican', 'justice', 'one', 'supreme']

✨Topic 1✨

['NUM', 'new', 'years', 'two', 'last', 'year', 'city', 'york', 'ago', 'country', 'week', 'since', 'people', 'police', 'three', 'month', 'first', 'one', 'million', 'percent']

✨Topic 2✨

['part', 'article', 'sign', 'newsletter', 'receive', 'black', 'american', 'times', 'war', 'women', 'thursdays', 'tuesdays', 'debatable', 'big', 'every', 'world', 'news', 'today', 'one', 'america']

✨Topic 3✨

['time', 'like', 'one', 'day', 'would', 'home', 'know', 'back', 'never', 'family', 'said', 'people', 'get', 'school', 'life', 'first', 'days', 'even', 'way', 'work']

✨Topic 4✨

['coronavirus', 'pandemic', 'states', 'NUM', 'covid', 'united', 'people', 'many', 'health', 'americans', 'country', 'crisis', 'america', 'public', 'world', 'even', 'economic', 'care', 'could', '